### 公众号：数据不吹牛
### 更多案例和有趣分析等你来撩

In [15]:
import os
import pandas as pd
import numpy as np
import time

In [2]:
os.chdir('C:\\Users\\Administrator\\教程\\知乎清洗')

### 打开第一个问题的回答

In [3]:
df = pd.read_excel('过年工作问题.xlsx')

In [4]:
df.head(2)

,token,内容,创建时间,性别,是否可以评论,更新时间,点赞数,用户ID,用户名,用户类型,签名,被关注人数,评论数,问题,专业,教育经历,行业
0,sun-wu-jing-91-57,不该。<br/>千万别说。<br/><br/>以我为例，<br/><br/>我实在是没法跟亲...,1518103736,1,True,1518103812,1342,ce8245b32b745f18ef58ca042a3a8651,孙悟净,people,她说要把它们都打败\n我说我会一直都在,37173,200,过年回家，亲戚问是做什么工作的，该说实话吗？,NaN,第二小学,"['黑色金属', '一无所有', '背负着空虚倔强']"
1,wo-ai-luo-4-14,"<figure><noscript><img src=""https://pic2.zhimg...",1518092713,1,True,1518092913,513,bef6149719717434b53d897031c5da20,林杰,people,鸡飞的男人,8,65,过年回家，亲戚问是做什么工作的，该说实话吗？,NaN,NaN,['创意艺术']


### 把代表性别的0，1，-1替换成女，男，未知

In [5]:
df['sex'] = np.where(df['性别'] == 1,'男',
                    np.where(df['性别'] == 0,'女','未知'))

In [6]:
df.head(2)

,token,内容,创建时间,性别,是否可以评论,更新时间,点赞数,用户ID,用户名,用户类型,签名,被关注人数,评论数,问题,专业,教育经历,行业,sex
0,sun-wu-jing-91-57,不该。<br/>千万别说。<br/><br/>以我为例，<br/><br/>我实在是没法跟亲...,1518103736,1,True,1518103812,1342,ce8245b32b745f18ef58ca042a3a8651,孙悟净,people,她说要把它们都打败\n我说我会一直都在,37173,200,过年回家，亲戚问是做什么工作的，该说实话吗？,NaN,第二小学,"['黑色金属', '一无所有', '背负着空虚倔强']",男
1,wo-ai-luo-4-14,"<figure><noscript><img src=""https://pic2.zhimg...",1518092713,1,True,1518092913,513,bef6149719717434b53d897031c5da20,林杰,people,鸡飞的男人,8,65,过年回家，亲戚问是做什么工作的，该说实话吗？,NaN,NaN,['创意艺术'],男


### 绘制匿名用户占比和剔除掉性别的占比图

In [6]:
from pyecharts import Pie

values1 = [len(df.loc[df['用户名'] == '匿名用户',])/len(df),1 - len(df.loc[df['用户名'] == '匿名用户',])/len(df)]
labels1 = ['匿名用户','正常用户']


values2 = [len(df.loc[df['用户名'] != '匿名用户',:].loc[df.loc[df['用户名'] != '匿名用户',:]['sex'] == '男',]),
           len(df.loc[df['sex'] == '女',]),len(df.loc[df['sex'] == '未知',])]
labels2 = ['男','女','未知']


pie = Pie('匿名情况和性别占比',title_pos = 'center',width = 1000)

pie.add('A',labels1,values1,center = [30,50],rosetype = 'radius',
        radius = [30,60],is_legend_show = False,is_label_show = True)

pie.add('B',labels2,values2,center = [75,50],rosetype = 'area',
        radius = [30,60],is_legend_show = False,is_label_show = True,
        is_random = True)

## 将时间戳转换为易读的时间格式
## 并绘制回答创建时间分布图

In [7]:
import datetime

def convert_time(x):
    return datetime.datetime.fromtimestamp(x).strftime('%Y-%m-%d')

#把更新时间，创建时间都转化成我们习惯的时间格式
df['更新时间'] = pd.to_datetime(df['更新时间'].apply(convert_time))
df['创建时间'] =pd.to_datetime(df['创建时间'].apply(convert_time))

from pyecharts import Bar

x = df.groupby('创建时间').count().index
y = df.groupby('创建时间').count()['用户名']

bar = Bar('回答创建时间分布',title_pos = 'center',width = 1000)
bar.add('A',x,y,is_legend_show = False,mark_point = ['max'])

### 由于匿名用户都不显示真实的行业，所以先剔除之

In [8]:
filter_result = df.loc[df['用户名'] != '匿名用户',:]
filter_result.head(2)

,token,内容,创建时间,性别,是否可以评论,更新时间,点赞数,用户ID,用户名,用户类型,签名,被关注人数,评论数,问题,专业,教育经历,行业,sex
0,sun-wu-jing-91-57,不该。<br/>千万别说。<br/><br/>以我为例，<br/><br/>我实在是没法跟亲...,2018-02-08,1,True,2018-02-08,1342,ce8245b32b745f18ef58ca042a3a8651,孙悟净,people,她说要把它们都打败\n我说我会一直都在,37173,200,过年回家，亲戚问是做什么工作的，该说实话吗？,NaN,第二小学,"['黑色金属', '一无所有', '背负着空虚倔强']",男
1,wo-ai-luo-4-14,"<figure><noscript><img src=""https://pic2.zhimg...",2018-02-08,1,True,2018-02-08,513,bef6149719717434b53d897031c5da20,林杰,people,鸡飞的男人,8,65,过年回家，亲戚问是做什么工作的，该说实话吗？,NaN,NaN,['创意艺术'],男


### 粗暴的提取行业相关信息

In [11]:
def return_hangye(x):
    if len(x) == 0:
        return '未知'
    else:
        return x[0][1:-1]
    
def return_pos(x):
    if len(x) <=1:
        return '未知'
    else:
        return x[-1][2:-1]

filter_result['所处行业'] = filter_result['行业'].astype(str).str[1:-1].replace("'","").str.split(',').apply(return_hangye)
filter_result['岗位'] = filter_result['行业'].astype(str).str[1:-1].replace("'","").str.split(',').apply(return_pos)

D:\python\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
D:\python\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [12]:
filter_result.head(1)

,token,内容,创建时间,性别,是否可以评论,更新时间,点赞数,用户ID,用户名,用户类型,签名,被关注人数,评论数,问题,专业,教育经历,行业,sex,所处行业,岗位
0,sun-wu-jing-91-57,不该。<br/>千万别说。<br/><br/>以我为例，<br/><br/>我实在是没法跟亲...,2018-02-08,1,True,2018-02-08,1342,ce8245b32b745f18ef58ca042a3a8651,孙悟净,people,她说要把它们都打败\n我说我会一直都在,37173,200,过年回家，亲戚问是做什么工作的，该说实话吗？,NaN,第二小学,"['黑色金属', '一无所有', '背负着空虚倔强']",男,黑色金属,背负着空虚倔强


### 可视化行业

#

In [13]:
tc = filter_result['所处行业'].value_counts()[1:11].reset_index()
tc['占比'] = tc['所处行业']/tc['所处行业'].sum()

x = tc['index']
y = tc['占比'].values

bar = Bar('吐槽人行业TOP10',title_pos = 'center',width = 1000)
bar.add('A',x,y,is_legend_show = False,mark_point = ['max'])

### 文章最后梗，不建议尝试运行

In [ ]:
count = 1
ask = True

while ask == True:
    print('对象{}'.format(count))
    count += 1